<a href="https://colab.research.google.com/github/yadavrishikesh/BYM-Spatial-Gamma-Gamma-Model/blob/main/Portland_all_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import jax
import warnings
warnings.simplefilter('ignore')

import contextily as ctx
import geopandas as gpd
import jax
import numpy as np
import pandas as pd
from bayesnf.spatiotemporal import BayesianNeuralFieldMAP

# Function to run the model and return the predictions DataFrame
def run_bayesian_model(train_file, test_file, output_file):
    # Load the train dataset
    df_train = pd.read_csv(train_file, index_col=0, parse_dates=['datetime'])

    # Log-transform the response variable
    df_train_log = df_train.copy()
    df_train_log['response'] = np.log1p(df_train_log['response'])

    # Define the model
    model = BayesianNeuralFieldMAP(
        width=512,
        depth=2,
        freq='D',
        seasonality_periods=['W'],  # Weekly seasonality
        num_seasonal_harmonics=[2], # 2 harmonics
        feature_cols=['datetime', 'lat', 'lon', 'speed', 'occupancy'], # Features
        target_col='response',      # Target column
    )

    # Train the model
    model = model.fit(
        df_train,
        seed=jax.random.PRNGKey(0),
        ensemble_size=5,
        num_epochs=500,
    )

    # Load the test dataset
    df_test = pd.read_csv(test_file, index_col=0, parse_dates=['datetime'])

    # Clean the test data by removing rows with NA in the response column
    df_test_cleaned = df_test.dropna(subset=['response'])

    # Predict on test data
    yhat, yhat_quantiles = model.predict(df_test_cleaned, quantiles=(0.025, 0.5, 0.975))

    # Create the predictions DataFrame with required columns
    predictions_df = pd.DataFrame({
        'datetime': df_test_cleaned['datetime'],
        'True_Values': df_test_cleaned['response'],
        'Predicted_Mean': yhat_quantiles[1],  # 0.5 quantile (median)
        '0.025_Quantile': yhat_quantiles[0],
        '0.975_Quantile': yhat_quantiles[2]
    })

    # Save the DataFrame to CSV
    predictions_df.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")

    return predictions_df

# Example usage
predictions_df = run_bayesian_model(
    train_file='train_data_Portland_pred-type_spatIntpl.csv',
    test_file='test_data_Portland_pred-type_spatIntpl.csv',
    output_file='predictions_output.csv'
)

# Display the first few rows of the resulting predictions DataFrame
predictions_df.head()
